## Here I translate GCN in tensorflow to GCN in tensorflow.keras
### This is the current method i'm focusing on 

In [44]:
# dependencies
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers

In [45]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.0-rc2
2.2.4-tf


In [46]:
# methods are functions associated with a class
# the data are the attributes

In [ ]:
adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype(float) #takes the first 10,000 molecules

This class allows us to build the weight and biases for each convolutional layer and also each model. After building the biases in the build method, I will perform the matrix multiplications in the call method for each model. The cell below will iteratively call upon all layers, until the whole model is build. The model will then be compiled. 

In [41]:
from tensorflow.keras import layers
class MyLayer(layers.Layer):

    def __init__(self, output_dim, adjacency, features, weight_shape, bias_shape, **kwargs):
        self.output_dim = output_dim
        self.features = tf.placeholder(shape = [100,50,58], dtype = 'float64')
        self.adjacency = tf.placeholder(shape = [100,50,50], dtype = 'float64')
        self.weight_shape = weight_shape
        self.bias_shape = bias_shape
        super(MyLayer, self).__init__(**kwargs)

    # still have to add attention weights
    def build(self):
        num_atoms = int(self.features.shape[1])
        input_dim = int(self.features.shape[2])
        weight_shape = self.weight_shape
        bias_shape = self.bias_shape
        
        self.kernel = self.add_weight(name='kernel',
                                  shape=weight_shape,
                                  initializer='uniform',
                                  trainable=True)
        
        self.bias = self.add_weight(name='bias',
                                shape=(bias_shape),
                                initializer='zeros',
                                trainable=True)
    
    # Make sure to call the `build` method at the end
        super(MyLayer, self).build(input_shape)
    
    #here is where I will implement all models, which will be built in another script. 
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [43]:
#this is where all models will be built

model = tf.keras.Sequential()
num_atoms = int(features.shape[1])
input_dim = int(features.shape[2])
hidden_dim = [input_dim]
num_layers=3
for i in range(num_layers):
    hidden_dim.append(32)
    print(MyLayer(32,adj,features,[i,i+1],[i+1])    

